<a href="https://colab.research.google.com/github/emmettgb/Emmetts-DS-NoteBooks/blob/master/Opiates_on_Accidental_Deaths.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [147]:
# Import required packages
import os, sys
in_colab = 'google.colab' in sys.modules

if in_colab:
    ## Install packages in Colab
    !pip install --upgrade category_encoders eli5 pandas-profiling plotly
import category_encoders as ce
import eli5
import pandas as pd
import sklearn

# Check package versions
from distutils.version import StrictVersion
assert StrictVersion(ce.__version__) >= StrictVersion('2.0.0')
assert StrictVersion(eli5.__version__) >= StrictVersion('0.9.0')
assert StrictVersion(pd.__version__) >= StrictVersion('0.24.2')
assert StrictVersion(sklearn.__version__) >= StrictVersion('0.21.3')

Requirement already up-to-date: category_encoders in /usr/local/lib/python3.6/dist-packages (2.0.0)
Requirement already up-to-date: eli5 in /usr/local/lib/python3.6/dist-packages (0.10.0)
Requirement already up-to-date: pandas-profiling in /usr/local/lib/python3.6/dist-packages (2.3.0)
Requirement already up-to-date: plotly in /usr/local/lib/python3.6/dist-packages (4.1.0)


# Data Cleaning

In [0]:
# Read data
url = 'https://data.ct.gov/api/views/rybz-nyjw/rows.csv'
# SOURCE: Data.Gov
df = pd.read_csv(url)
dfc = df.copy()
df.head(5)
#Some imports out of the way
import scipy.stats
import numpy as np

In [0]:
def datacleaner3000(df):
  #Replace all Ys with integers,
  #Replace all Ns with 0s
  df = df.replace('Y',1)
  df = df.replace('N',0)
  #The Ns are actually stored as NaNs,
  #So we are going to replace the null values in the columns with 0
  #But first, Just in case
  df = df.replace('?',np.NaN)
  df['Heroin'] = df['Heroin'].replace(np.NaN,0)
  df['Cocaine'] = df['Cocaine'].replace(np.NaN,0)
  df['OtherSignifican'] = df['OtherSignifican'].replace(np.NaN,0)
  df['Fentanyl'] = df['Fentanyl'].replace(np.NaN,0)
  df['Oxycodone'] = df['Oxycodone'].replace(np.NaN,0)
  df = df.drop(columns='FentanylAnalogue')
  df['Ethanol'] = df['Ethanol'].replace(np.NaN,0)
  df['Oxymorphone'] = df['Oxymorphone'].replace(np.NaN,0)
  df['Methadone'] = df['Methadone'].replace(np.NaN,0)
  df['Amphet'] = df['Amphet'].replace(np.NaN,0)
  df['Tramad'] = df['Tramad'].replace(np.NaN,0)
  df = df.rename(columns={"Morphine_NotHeroin": "Morphine"})
  df['Hydromorphone'] = df['Hydromorphone'].replace(np.NaN,0)
  df['Hydrocodone'] = df['Hydrocodone'].replace(np.NaN,0)
  df['Benzodiazepine'] = df['Benzodiazepine'].replace(np.NaN,0)
  df['Morphine'] = df['Morphine'].replace(np.NaN,0)
  df['AnyOpioid'] = df['AnyOpioid'].replace(np.NaN,0)
  df = df.drop(columns = 'Other')
  df['Sex'] = df['Sex'].replace('Male',0)
  df['Sex'] = df['Sex'].replace('Female',1)
  return(df)

In [0]:
#Apply my lovely clean function:
df = datacleaner3000(df)

In [151]:
df = df.replace('Y',1)
df.head(5)

,ID,Date,DateType,Age,Sex,Race,ResidenceCity,ResidenceCounty,ResidenceState,DeathCity,DeathCounty,Location,LocationifOther,DescriptionofInjury,InjuryPlace,InjuryCity,InjuryCounty,InjuryState,COD,OtherSignifican,Heroin,Cocaine,Fentanyl,Oxycodone,Oxymorphone,Ethanol,Hydrocodone,Benzodiazepine,Methadone,Amphet,Tramad,Morphine,Hydromorphone,OpiateNOS,AnyOpioid,MannerofDeath,DeathCityGeo,ResidenceCityGeo,InjuryCityGeo
0,14-0273,06/28/2014 12:00:00 AM,DateReported,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,substance,NaN,NaN,NaN,NaN,"Acute fent, hydrocod, benzodiazepine",0,0.0,0.0,1,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0,0.0,NaN,0.0,Accident,"CT\n(41.575155, -72.738288)","CT\n(41.575155, -72.738288)","CT\n(41.575155, -72.738288)"
1,13-0102,03/21/2013 12:00:00 AM,DateofDeath,48.0,0,Black,NORWALK,NaN,NaN,NORWALK,FAIRFIELD,Hospital,NaN,NaN,NaN,NaN,NaN,NaN,Cocaine Intoxication,0,0.0,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,NaN,0.0,Accident,"Norwalk, CT\n(41.11805, -73.412906)","NORWALK, CT\n(41.11805, -73.412906)","CT\n(41.575155, -72.738288)"
2,16-0165,03/13/2016 12:00:00 AM,DateofDeath,30.0,1,White,SANDY HOOK,FAIRFIELD,CT,DANBURY,NaN,Hospital,NaN,Substance Abuse,Unknown,UNKNOWN,NaN,NaN,Acute Heroin and Cocaine Intoxication,0,1.0,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,NaN,1.0,Accident,"Danbury, CT\n(41.393666, -73.451539)","SANDY HOOK, CT\n(41.419998, -73.282501)",NaN
3,16-0208,03/31/2016 12:00:00 AM,DateofDeath,23.0,0,White,RYE,WESTCHESTER,NY,GREENWICH,NaN,Hospital,NaN,substance abuse,Residence,RYE,NaN,NaN,Acute Fentanyl and Morphine Intoxication,0,1.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,NaN,1.0,Accident,"Greenwich, CT\n(41.026526, -73.628549)",NaN,NaN
4,13-0052,02/13/2013 12:00:00 AM,DateofDeath,22.0,0,"Asian, Other",FLUSHING,QUEENS,NaN,GREENWICH,FAIRFIELD,Hospital,NaN,Transdermal Absorption,Other,NaN,NaN,NaN,Fentanyl Intoxication,0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,NaN,0.0,Accident,"Greenwich, CT\n(41.026526, -73.628549)",NaN,"CT\n(41.575155, -72.738288)"


In [0]:
#We may need these later
from sklearn.model_selection import train_test_split
import category_encoders as ce
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline

# Train/Test Split

In [153]:
print(df['Date'].value_counts)
#Test Before 2017 Looks good.

<bound method IndexOpsMixin.value_counts of 0       06/28/2014 12:00:00 AM
1       03/21/2013 12:00:00 AM
2       03/13/2016 12:00:00 AM
3       03/31/2016 12:00:00 AM
4       02/13/2013 12:00:00 AM
5       06/29/2014 12:00:00 AM
6       08/12/2012 12:00:00 AM
7       11/10/2013 12:00:00 AM
8       04/25/2012 12:00:00 AM
9       05/15/2013 12:00:00 AM
10      08/23/2012 12:00:00 AM
11      07/05/2015 12:00:00 AM
12                         NaN
13      05/14/2015 12:00:00 AM
14      01/13/2016 12:00:00 AM
15      08/19/2013 12:00:00 AM
16      01/29/2014 12:00:00 AM
17      03/02/2012 12:00:00 AM
18      01/30/2016 12:00:00 AM
19      12/20/2016 12:00:00 AM
20      11/14/2014 12:00:00 AM
21      06/02/2015 12:00:00 AM
22      05/02/2014 12:00:00 AM
23      10/07/2016 12:00:00 AM
24      07/16/2016 12:00:00 AM
25      10/13/2017 12:00:00 AM
26      02/10/2018 12:00:00 AM
27      04/12/2015 12:00:00 AM
28      02/27/2015 12:00:00 AM
29      04/11/2018 12:00:00 AM
                 ...      

In [154]:
df['Date'] = pd.to_datetime(df['Date'])
#Masks to split DataFrame
final_date = '01-01-2018'
final_data = '01-30-2017'
early_data = '01-01-2016'
dfor = df.copy()
stockmask = dfor['Date'] >= early_data
stockmask2 = dfor['Date'] >= final_date
dfor = df[stockmask]
dffor = df[stockmask2]
dfor.groupby(dfor.Date.dt.year)
testmask = df['Date'] >= final_date
trainmask = df['Date'] <= final_date
dfor['Date'] = dfor['Date'].astype(str)
dfor['AnyOpioid']=dfor['AnyOpioid'].astype(int)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [0]:
df['Date'] = df['Date'].astype(str)
test = df[testmask]
train = df[trainmask]

In [0]:
target =  'AnyOpioid'
X_tr = train.drop(columns=target)
y_tr = train[target]
X_te = test.drop(columns=target)
y_te = test[target]

# pipeline model

In [0]:
#Lol see you in an hour google colab
pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    IterativeImputer(), 
    RandomForestClassifier(n_estimators=2500, random_state=35, n_jobs=-3)
)

In [158]:
pip install plotly

# Visual Modeling

In [159]:
import plotly.express as px
import plotly.graph_objects as go
fig = px.scatter(dfor, x="Date", y='DescriptionofInjury', color="Age",
                 size='AnyOpioid',
                 title='Accidental Drug-Related Deaths by Cause')
fig.update_layout(
    xaxis=go.layout.XAxis(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)
fig.show()

In [0]:
#Pull out data with mask so that I can extrapalate a difference
opimask = dfor['AnyOpioid'] == 1
opinull = dfor['AnyOpioid'] == 0
opidf = dfor[opimask]
opinull = dfor[opinull]

In [199]:
print(len(opidf['ID']))
print(len(opinull['ID']))

948
70


In [161]:
fig2 = px.scatter(opidf, x="Date", y="Location", color="Location", 
    marginal_x="histogram",
    title="Accidental Deaths from Opiates by location and Date")
fig2.show()

In [196]:
fig3 = px.scatter(opinull, x="Date",
                  y="Location", color="Location",
                  marginal_x="histogram",
                  title="Accidental Deaths not involving opiates")
fig3.show()

# Statistics

In [0]:
#opimask = dfor['AnyOpioid'] == 1
#opinull = dfor['AnyOpioid'] == 0
#P = prob, A = prior, B = Evidence, 
def bay_ther(p,a,b):
  psterior = (p*(b|a) * p*(a)) / (p*b)
  return(psterior)

In [164]:
print(df['AnyOpioid'].value_counts(normalize=True) * 100)

0.0    51.811949
1.0    48.188051
Name: AnyOpioid, dtype: float64


In [165]:
print(df['AnyOpioid'].value_counts(normalize=True) * 100)

0.0    51.811949
1.0    48.188051
Name: AnyOpioid, dtype: float64


In [166]:
bay_ther(p=100,a=51,b=48)

5418.75

In [167]:
total = len(df['ID'])
print(total)

5105


In [168]:
cocmask = df['Cocaine'] == 1
cocp = df[cocmask]
coctotal = len(cocp['ID'])
print(coctotal)

1521


In [169]:
fentmask = df['Fentanyl'] == 1
fentp = df[fentmask]
fenttotal = len(fentp['Fentanyl'])
print(fenttotal)

2228


In [170]:
hermask = df['Heroin'] == 1
herp = df[hermask]
hertotal = len(herp['Heroin'])
print(hertotal)

2529


In [171]:
print(2228+2529)

4757


In [0]:
#Fentanol caused more Drug-Related accidental deaths than all Opiates besides
#Fentanol and heroine caused more death than any other substance

# ML  Fitting and Visualization

In [173]:
pipeline.fit(X_tr,y_tr)

Pipeline(memory=None,
         steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['ID', 'Date', 'DateType', 'Sex', 'Race',
                                      'ResidenceCity', 'ResidenceCounty',
                                      'ResidenceState', 'DeathCity',
                                      'DeathCounty', 'Location',
                                      'LocationifOther', 'DescriptionofInjury',
                                      'InjuryPlace', 'InjuryCity',
                                      'InjuryCounty', 'InjuryState', 'COD',
                                      'OtherSignifican', 'Fentanyl', 'Morphine',
                                      'MannerofDeath', '...
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                     

In [0]:
tntpreddf = df.copy()
y_pred = pipeline.predict(X_te)

In [175]:
print(y_pred)

[0. 0. 0. ... 0. 0. 1.]


In [176]:
test.head(2)

,ID,Date,DateType,Age,Sex,Race,ResidenceCity,ResidenceCounty,ResidenceState,DeathCity,DeathCounty,Location,LocationifOther,DescriptionofInjury,InjuryPlace,InjuryCity,InjuryCounty,InjuryState,COD,OtherSignifican,Heroin,Cocaine,Fentanyl,Oxycodone,Oxymorphone,Ethanol,Hydrocodone,Benzodiazepine,Methadone,Amphet,Tramad,Morphine,Hydromorphone,OpiateNOS,AnyOpioid,MannerofDeath,DeathCityGeo,ResidenceCityGeo,InjuryCityGeo
26,18-0095,2018-02-10,DateReported,51.0,0,White,HARTFORD,HARTFORD,CT,HARTFORD,HARTFORD,Hospital,NaN,Drug Use,"Other, Public Buildings",HARTFORD,HARTFORD,NaN,"Acute IntoxicationCombined Effects of Ethanol,...",0,0.0,0.0,1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0,0.0,NaN,1.0,Accident,"Hartford, CT\n(41.765775, -72.673356)","HARTFORD, CT\n(41.765775, -72.673356)","HARTFORD, CT\n(41.765775, -72.673356)"
29,18-0285,2018-04-11,DateReported,25.0,0,"Hispanic, White",HARRISBURG,DAUPHIN,PA,NEW HAVEN,NEW HAVEN,Nursing Home,NaN,Substance Abuse,Unknown,NaN,NaN,NaN,Complications of Anoxic-Ischemic Encephalopath...,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,1.0,1.0,Accident,"NEW HAVEN, CT\n(41.308252, -72.924161)","HARRISBURG, CT\n(41.617199, -73.284203)","CT\n(41.575155, -72.738288)"


In [0]:
test = df[testmask]
train = df[trainmask]

In [191]:
test['Opioid-Based'] = y_pred

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [195]:
fig = px.scatter(test, x="Date", y='ID', color="Opioid-Based",
                 size='AnyOpioid',
                title='Opiate Versus non-Opiate related death prediction: 2019')
fig.show()